# --- Day 10: Factory ---


In [1]:
# --- Support Functions ---
# This section contains support functions used by the main code.

import re
import itertools
from typing import List, Tuple, Set, Dict

def read_input(file_path):
    with open(file_path, 'r') as file:
        return file.read().splitlines()

class Machine:
    def __init__(self, line: str):
        self.target, self.buttons, self.joltage = self._parse_machine(line)
        self.nlights = len(self.target)      # number of light of the indicator
        self.nbuttons = len(self.buttons)   # number of buttons
        self.button_vectors = self._make_vectors()
        self.best = None
        self.best_combo = None

    def _parse_machine(self, line):
        line = line.strip()
        # Lights Indicator pattern
        lights_string = re.search(r"\[(.*?)\]", line).group(1)
        target = [1 if c == '#' else 0 for c in lights_string]

        # Buttons
        button_strings = re.findall(r"\((.*?)\)", line)
        buttons = []
        for b in button_strings:
            b = b.strip()
            buttons.append([int(x) for x in b.split(",")])

        # Joltage
        joltage_string = re.search(r"\{(.*?)\}", line).group(1)
        joltage = [int(x) for x in joltage_string.split(",")]

        return target, buttons, joltage

    def _make_vectors(self):
        """Convert button toggle definition in boolean vectors"""
        vectors = []
        for button in self.buttons:
            vec = [0] * self.nlights
            for i in button:
                vec[i] = 1
            vectors.append(vec)
        return vectors

    def min_presses(self):
        """Solve via brute-force over 2^m subsets (fine for ~20 buttons)."""
        best = None

        for mask in range(1 << self.nbuttons):
            presses = mask.bit_count()

            # prune if already worse
            if best is not None and presses >= best:
                continue

            # compute results of pressing buttons in mask
            result = [0] * self.nlights
            for i in range(self.nbuttons):
                if (mask >> i) & 1:
                    for j in range(self.nlights):
                        result[j] ^= self.button_vectors[i][j]

            if result == self.target:
                best = presses

        return best



In [2]:
# --- Part ONE ---

input = read_input('input.txt')

result = 0

machines = []
for l in input:
    machine = Machine(l)
    result += machine.min_presses()


print("Part ONE:", result)

Part ONE: 449


In [3]:
# --- Part 2 Support Functions ---
# This section contains support functions used by the main code.

# For Part Two, using brute force is not feasible because 
# the solution space grows exponentially.
# So we use a CP-Sat (Constrain Programming with Satisfability) algorithm aproach
# see: https://developers.google.com/optimization/cp/cp_solver

!pip install -q ortools

from ortools.sat.python import cp_model

def solve_machine(buttons: List[Tuple[int, ...]], joltage: List[int]):
    n = len (joltage) # number of counters 
    m = len (buttons) # number of buttons

    model = cp_model.CpModel()


    # Upper bound
    # For each button, determine the maximum feasible multiplier
    # depending on the minimum target value of the counters it modifies
    max_presses = [min([joltage[c] for c in range(n) if c in btn]) for btn in buttons]

    # problem variables: x_j  that is number of presses for button j
    x = [
        model.NewIntVar(0,max_presses[j], f"x_{j}")
        for j in range(m)
    ]

    # Counter constraint
    # This is what we want to obtain : each counter must be as
    # declared in joltage
    for i in range(n):
        model.Add(
            sum(x[j] for j in range(m) if i in buttons[j]) == joltage[i]
        )

    # Objective
    model.Minimize(sum(x))

    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 10 # limit time
    solver.parameters.num_search_workers = 8

    status = solver.Solve(model)

    if status not in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        raise RuntimeError("No solution found")
    
    solution = [solver.value(x[j]) for j in range(m)]

    return solution, sum(solution)
    


In [4]:
# --- Part TWO ---


input = read_input('input.txt')

result = 0

i = 1

machines = []
for l in input:
    m = Machine(l)
    machines.append(m)
    solution, presses = solve_machine(m.buttons, m.joltage)

    # print (f"Presses : {presses} , Solution : {solution}")

    result += presses

print("Part TWO:", result)

Part TWO: 17848
